In [15]:
library(dplyr)
library(parallel)
library(stringr)

reprocess_switch=0 #0 or 1. 1 means to recreate all reach and node products following some change in processing. 
#0 means to just append to existing dataframes

# hubname='UNC'
# rivername='WK'
# continent='oc'
# PT_key_file= 'WK_Key_20230331.csv' #WK
# utm_zone='58 +south'#WK='58 +south'

# hubname='UMass'
# rivername='CR'
# continent='na'
# PT_key_file='Key_CR_FS_20230322.csv' #CT
# utm_zone=18 #Ct= 18

hubname='CU'
rivername='WM'
continent='na'
PT_key_file= 'SWOTCalVal_WM_KEY_20230326_20230510.csv' #WM
utm_zone=10 #WM= 10


buffer=500 #m, 'extends' the reach

setwd(paste0('/nas/cee-water/cjgleason/calval/Processed data/',hubname,'/'))
domain_file=keyfile=paste0(rivername,'_domain.csv')


#PT paths---------
PT_data_directory=paste0('/nas/cee-water/cjgleason/calval/xml_scripts/',hubname,'/Munged/')
                        
flagged_PT_output_directory='Flagged PT/'
#--------------------------------------------------
#drift paths------------------------------------------
GNSS_drift_data_directory=paste0('From Andy/',hubname,'_netCDFs/')
if(reprocess_switch==1){
    
    drift_string= paste0('Munged drifts/','reprocessed_',  str_replace_all(as.character(Sys.Date()),'\\-','_'))
    PT_string =paste0('Munged PT/','reprocessed_',  str_replace_all(as.character(Sys.Date()),'\\-','_'))
    reachnode_string= paste0('Data frames/','reprocessed_',  str_replace_all(as.character(Sys.Date()),'\\-','_'))
    node_string =paste0('Data frames/','reprocessed_',  str_replace_all(as.character(Sys.Date()),'\\-','_'),'/node')
    reach_string=paste0('Data frames/','reprocessed_',  str_replace_all(as.character(Sys.Date()),'\\-','_'),'/reach')
    
    #check if we've already reprocessed today
    if (dir.exists(drift_string)){
        #if we have, then use that as the output and clear the files in the drift directory
        unlink(drift_string, recursive = TRUE)
        unlink(PT_string,recursive = TRUE)
        dir.create(drift_string)
        dir.create(PT_string)
        QA_QC_drift_output_directory=paste0(drift_string,'/')
        reachnode_output_directory=paste0(reachnode_string,'/')
        } else {
        #if we haven't reprocessed today
    dir.create(drift_string)
    dir.create(PT_string)
    dir.create(reachnode_string)
    dir.create(node_string)
    dir.create(reach_string)
        
    QA_QC_drift_output_directory=paste0(drift_string,'/')
    reachnode_output_directory=paste0(reachnode_string,'/')
    QA_QC_PT_output_directory=paste0(PT_string,'/')
    }
    
    } else { #we aren't reprocessing. find the most recent folders to use
    
folderlist= list.files('Munged drifts',full.names = TRUE)
    
foldertimes=file.info(folderlist)%>%
 mutate(mintime= Sys.time()-mtime) %>%
 filter(mintime== min(mintime)) 
    
QA_QC_drift_output_directory=paste0(row.names(foldertimes),'/')  
    
folderlist2= list.files('Data frames',full.names = TRUE)
    
foldertimes2=file.info(folderlist2)%>%
 mutate(mintime= Sys.time()-mtime) %>%
 filter(mintime== min(mintime)) 
    
reachnode_output_directory=paste0(row.names(foldertimes2),'/')
    
folderlist3= list.files('Munged PT',full.names = TRUE)
    
foldertimes3=file.info(folderlist3)%>%
 mutate(mintime= Sys.time()-mtime) %>%
 filter(mintime== min(mintime))  
    
QA_QC_PT_output_directory=paste0(row.names(foldertimes3),'/') 
}

flagged_drift_output_directory='Flagged drifts/'
#--------------------------------------------------

#sword paths----------------------------------------
SWORD_path=paste0('/nas/cee-water/cjgleason/calval/SWORD_15/netcdf/',continent,
                  '_sword_v15.nc')
#------------------------------



In [ ]:
QA_QC_drift_output_directory
QA_QC_PT_output_directory
reachnode_output_directory

In [ ]:
#create dataframes from drifts---------------------------------------------------------
#pull filename before the .csv
raw_GNSS=sub( "\\..*","", list.files(GNSS_drift_data_directory))

#pull filename before the second _
QA_QC_drifts=sub( "\\..*","",list.files(QA_QC_drift_output_directory))
flagged_drifts=sub("\\..*","",list.files(flagged_drift_output_directory))
#what raw drift data have not been munged
unmunged_drifts=setdiff(raw_GNSS,c(flagged_drifts,QA_QC_drifts))



if(!identical(unmunged_drifts,character(0))){
  source('/nas/cee-water/cjgleason/calval_toolbox/R code/create_GNSS_dataframe.R')
   cl=makeCluster(44)

    
  dummy=parLapply(cl=cl,unmunged_drifts,create_gnss_dataframe,
                  gnss_drift_data_directory=GNSS_drift_data_directory,
                  output_directory=QA_QC_drift_output_directory)
  stopCluster(cl)
}

#-------------------------------------------------

In [88]:
#munge PTs if needed------
dist_thresh=150 # 150m
time_thresh= 15*60 #minutes as seconds, centered, so 15 =30 mins total time
GNSS_sd_thresh=0.15 # 15cm how much variance do you want in the GNSS data when it is within the distance threshold?
offset_sd_thresh=0.10 #m, so 10cm. the the PT apparantly shift by more than a cm?
change_thresh_15_min=0.05#m- does it change more than 5cm in 15 minutes? that is a discontinuity in offset

#check for un-munged PT data
#first, move .csv files with an 'L1' in them over to the PT_data_directory
munged_files= list.files(PT_data_directory,
                         recursive= TRUE)

PT_index=which(!is.na(do.call(rbind,lapply(munged_files,str_match,'PT_L1'))))
PT_files=munged_files[PT_index]
csv_index=which(!is.na(do.call(rbind,lapply(PT_files,str_match,'.csv'))))
csv_PT_files=PT_files[csv_index]


#pull filename before the .csv
raw_PT=sub("\\..*","",csv_PT_files)
raw_PT=sub(".*/","",raw_PT)
#pull filenam\\..*e before the second _
QA_QC_PTs=sub("\\..*","",list.files(QA_QC_PT_output_directory))
flagged_PTs=sub( "\\..*","",list.files(flagged_PT_output_directory))
#what raw PT data have not been munged
unmunged_PTs=paste0(setdiff(raw_PT,c(flagged_PTs,QA_QC_PTs)),'.csv')
#run the PTs that are not yet munged

files_to_run=list()
 for (i in 1:length(unmunged_PTs)){
files_to_run[[i]]=munged_files[which(!is.na(str_match(munged_files,unmunged_PTs[i])))]
     }

 files_to_run=paste0('/nas/cee-water/cjgleason/calval/xml_scripts/',hubname,'/Munged/',files_to_run)

if(!identical(unmunged_PTs,character(0))){
  source('/nas/cee-water/cjgleason/calval_toolbox/R code/correct_PT_to_GNSS.R')

      cl=makeCluster(44)
  
  dummy=parLapply(cl, files_to_run,correct_pt_to_gnss,
                  pt_key_file=PT_key_file,
                  dist_thresh=dist_thresh,
                  time_thresh=time_thresh,
                  pt_data_directory=PT_data_directory,
                  gnss_drift_data_directory=QA_QC_drift_output_directory,
                  QA_QC_pt_output_directory=QA_QC_PT_output_directory,
                  flagged_pt_output_directory=flagged_PT_output_directory,
                  gnss_sd_thresh=GNSS_sd_thresh,
                  offset_sd_thresh=offset_sd_thresh,
                  change_thresh_15_min=change_thresh_15_min)
  
  stopCluster(cl)
}

#-----------------------------


In [ ]:
#calculate slopes and heights from drifts within nodes and reaches------

SWORD_reach= read.csv(domain_file)
this_river_reach_IDs= as.numeric(unique(SWORD_reach$Reach_ID[!is.na(SWORD_reach$Reach_ID)]))
this_river_node_IDs= as.numeric(unique(SWORD_reach$Node_ID[!is.na(SWORD_reach$Node_ID)]))
#photo_path= 'Willamette/Watermask/Demo_output/Shapefile/'
#this_river_reach_IDs
options(warn=-1)
source('/nas/cee-water/cjgleason/calval_toolbox/R code/calculate_slope_wse_fromdrift.R')

dummy=calculate_slope_wse_fromdrift(SWORD_path=SWORD_path,
                                    drift_directory=QA_QC_drift_output_directory,
                                    PT_directory=PT_directory,
                                    output_directory=reachnode_output_directory,
                                    this_river_reach_ids=this_river_reach_IDs,
                                    this_river_node_ids=this_river_node_IDs,
                                    utm_zone=utm_zone, 
                                    buffer=buffer,
                                    rivername=rivername,
                                    reprocess_switch=reprocess_switch)

# dummy
#-----------------------------

In [89]:
#calculate slopes and heights from PTs within nodes and reaches----
PT_files=paste0(QA_QC_PT_output_directory,list.files(QA_QC_PT_output_directory))
SWORD_reach= read.csv(domain_file)
this_river_reach_IDs= as.numeric(as.character(unique(SWORD_reach$Reach_ID)))


alongstream_error= 0.0001*200 #m error we get from the downstream slope of a reach in a node. This placeholder is a 1e-4 slope over a 200m node
crossstream_error= 0.005 #m error we get from PT not representing cross stream superelevation/noise in a node
measurement_error= 0.001 #m error we get from PT measurement itself 
source('/nas/cee-water/cjgleason/calval_toolbox/R code/calculate_slope_wse_fromPT.R')


dummy=calculate_slope_wse_fromPT(keyfile=PT_key_file,
                                 pt_files=PT_files,
                                 SWORD_path=SWORD_path,
                                 SWORD_reach=SWORD_reach,
                                 this_river_reach_ids=this_river_reach_IDs,
                                 rivername=rivername,
                                 output_directory= reachnode_output_directory,
                                 alongstream_error=alongstream_error,
                                 crossstream_error=crossstream_error,
                                 measurement_error=measurement_error)
#-----------------------------

[1] ">>>> WARNING <<<  attribute _FillValue is an 8-byte value, but R"
[1] "does not support this data type. I am returning a double precision"
[1] "floating point, but you must be aware that this could lose precision!"
[1] ">>>> WARNING <<<  attribute _FillValue is an 8-byte value, but R"
[1] "does not support this data type. I am returning a double precision"
[1] "floating point, but you must be aware that this could lose precision!"
[1] ">>>> WARNING <<<  attribute _FillValue is an 8-byte value, but R"
[1] "does not support this data type. I am returning a double precision"
[1] "floating point, but you must be aware that this could lose precision!"
[1] ">>>> WARNING <<<  attribute _FillValue is an 8-byte value, but R"
[1] "does not support this data type. I am returning a double precision"
[1] "floating point, but you must be aware that this could lose precision!"
[1] ">>>> WARNING <<<  attribute _FillValue is an 8-byte value, but R"
[1] "does not support this data type. I am return

`summarise()` has grouped output by 'node_id'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'reach_id'. You can override using the
`.groups` argument.
Warning message in left_join(., us_df, by = c("reach_id", "pt_time_UTC", "upstream_node")):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”
Warning message in left_join(., ds_df, by = c("reach_id", "pt_time_UTC", "downstream_node")):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 55522 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [ ]:
# passfile='/nas/cee-water/cjgleason/calval/Processed data/riversp_list_clean.txt'
# passnames=read.delim(passfile,header=F)$V1[which(!is.na(str_match(read.delim(passfile,header=F)$V1,'NA')))]

# time_threshold_sec= 120*60 #two hour
# wse_threshold_m=0.05 #within 5cm
# distance_threshold_m =200 #within 200m

# if (!dir.exists(paste0('Matched_drifts/','processed_',str_replace_all(as.character(Sys.Date()),'\\-','_')))){
#     dir.create(paste0('Matched_drifts/','processed_',str_replace_all(as.character(Sys.Date()),'\\-','_')))}


# matched_output_directory= paste0('Matched_drifts/','processed_',str_replace_all(as.character(Sys.Date()),'\\-','_'),'/')
# munged_drift_directory= QA_QC_drift_output_directory
# munged_pt_directory=QA_QC_PT_output_directory

# source('/nas/cee-water/cjgleason/calval_toolbox/R code/select_appropriate_drift.R')



# dummy=select_appropriate_drift(passnames[1],
#                                drift_node_directory= paste0(reachnode_output_directory,'/node/'),
#                                munged_drift_directory=munged_drift_directory,
#                                matched_output_directory=matched_output_directory,
#                                munged_pt_directory=munged_pt_directory,
#                                time_threshold_sec=time_threshold_sec,
#                                wse_threshold_m= wse_threshold_m,
#                                distance_threshold_m=distance_threshold_m,
#                                keyfile=PT_key_file,
#                                rivername=rivername)


In [ ]:
#define what drift goes with what SWOT overpass--------------------
#SWOT_L2_HR_RiverSP_<FileIdentifier>_<CycleID>_<PassID>_<ContinentID>_<RangeBeginningDateTime>_<RangeEndingDateTime>_<CRID>_<ProductCounter>.<extension> 

passfile='/nas/cee-water/cjgleason/calval/Processed data/riversp_list_clean.txt'
passnames=read.delim(passfile,header=F)$V1[which(!is.na(str_match(read.delim(passfile,header=F)$V1,'NA')))]

time_threshold_sec= 120*60 #two hour
wse_threshold_m=0.05 #within 5cm
distance_threshold_m =200 #within 200m

if (!dir.exists(paste0('Matched_drifts/','processed_',str_replace_all(as.character(Sys.Date()),'\\-','_')))){
    dir.create(paste0('Matched_drifts/','processed_',str_replace_all(as.character(Sys.Date()),'\\-','_')))}


matched_output_directory= paste0('Matched_drifts/','processed_',str_replace_all(as.character(Sys.Date()),'\\-','_'),'/')
munged_drift_directory= QA_QC_drift_output_directory
munged_pt_directory=QA_QC_PT_output_directory

source('/nas/cee-water/cjgleason/calval_toolbox/R code/select_appropriate_drift.R')


  
  cl=makeCluster(44)
dummy=parLapply(cl, passnames,select_appropriate_drift,
                               drift_node_directory= paste0(reachnode_output_directory,'/node/'),
                               munged_drift_directory=munged_drift_directory,
                               matched_output_directory=matched_output_directory,
                               munged_pt_directory=munged_pt_directory,
                               time_threshold_sec=time_threshold_sec,
                               wse_threshold_m= wse_threshold_m,
                               distance_threshold_m=distance_threshold_m,
                               keyfile=PT_key_file,
                               rivername=rivername)

stopCluster(cl)
#-----------------------------

In [ ]:
#calcluate areas from images------------------
a=Sys.time()

library(parallel)
source('/nas/cee-water/cjgleason/calval_toolbox/R code/calculate_area_from_imagery.R')

utm_zone = 10
#scale max width (in cases of under/overestimated or centerline offset)
scale_maxwidth = 3
datetime=as.POSIXct('2022-07-29 19:00:00')
##other parameter setting
#Threshold for water mapping
water_index_threshold = 0.2
reach_or_node='node'

#path of input image 
image_list = paste0('/nas/cee-water/cjgleason/calval/willamette_geotiffs/raw/WillametteROI/',
                    list.files('/nas/cee-water/cjgleason/calval/willamette_geotiffs/raw/WillametteROI/'))

#path of input SWORD data (netcdf)
SWORD_path='/nas/cee-water/cjgleason/SWORDv14/Reaches_Nodes/netcdf/na_sword_v11.nc'
#output dir
dir_output = '/nas/cee-water/cjgleason/calval_toolbox/Willamette/Watermask/'

# #selected reach and nodes to be processed
this_river_reach_ids <- c(78220000191,78220000181,78220000171)
this_river_node_ids <- c(78220000190011,78220000180811,78220000180661,78220000180511,
                        78220000180351,78220000180341,78220000180201,
                         78220000180191,78220000170571,78220000170401,78220000170281,78220000170161,78220000170011)

reach=c(78220000191,
        78220000181,78220000181,78220000181,78220000181,78220000181,78220000181,78220000181,
        78220000171,78220000171,78220000171,78220000171,78220000171,
        78220000191,78220000181,78220000171)
        
node=c(78220000190011, 
       78220000180811,78220000180661,78220000180511,78220000180351,78220000180341,78220000180201,78220000180191,
       78220000170571,78220000170401,78220000170281,78220000170161,78220000170011,
       'NA','NA','NA')

reach_or_node=c(rep('node',times=13),rep('reach',times=3))

# reach=c(78220000191)
        
# node=c(78220000190011)

# reach_or_node='node'


df=data.frame(reach=reach,node=node,reach_or_node=reach_or_node)
 input_list=do.call("mapply", c(list, df, SIMPLIFY = FALSE, USE.NAMES=FALSE))
#print(input_list)
#  cl=makeCluster(16)
# dummy=do.call(rbind,parLapply(cl,input_list,calculate_area_from_imagery,image_list=image_list,
#                                      utm_zone=utm_zone,
#                                      datetime=datetime,
#                                      scale_maxwidth=scale_maxwidth, 
#                                      SWORD_path=SWORD_path,
#                                      water_index_threshold=water_index_threshold,
#                                     dir_output=dir_output  ))
#  write.csv(dummy,paste0(dir_output,'CSV/','willamete_epxeriment_area_df.csv'))
# stopCluster(cl)



calculate_area_from_imagery(input_list=input_list[[1]],
                            image_list=image_list[[1]],
                                     utm_zone=utm_zone,
                                     datetime=datetime,
                                     scale_maxwidth=scale_maxwidth, 
                                     SWORD_path=SWORD_path,
                                     water_index_threshold=water_index_threshold,
                                    dir_output=dir_output  )
print(Sys.time()-a)#calcluate areas from images----
